In [1]:
import scipy.stats as stats
import os
import pandas as pd
import numpy as np
from time import time

In [3]:
def GetExpectedProbability(counts:pd.DataFrame, ax:int) -> pd.Series:
    
    '''
    This functions computes the expected values shown as probabilities given a contingency table
    
    Parameters
    ----------
    
    counts: pd.DataFrame
        Pandas DataFrame containing the counts of each group
    
    ax: int
        Integer 0 or 1 indicating along which axis will be used to compute the expected value
    
    Return
    ------
    
    pd.Series: pandas Series indicating the probabilities for every group
    
    '''
    
    total_sums = counts.sum(axis=ax)
    
    return total_sums / total_sums.sum()

In [ ]:
def BinaryMetrics(A:set,B:set) -> pd.DataFrame:

    '''
    
    This function computes the True positives (TP), False Negatives (FN) and False Positives (FP)
    from the comparation between the interactions found in two networks
    
    Parameters
    ----------

    A: set
        Reference set to calculate True Positives (TP) and False Negatives (FN) 
    
    B: set
        Complement set to calculate True Positives (TP) and False Positives (FP)
        
    Returns
    -------
    
        pd.DataFrame: A data frame containing the TP, FP, FN of the networks compared

    '''

    TP = len(A.intersection(B))
    FP = len(B.difference(A))
    FN = len(A.difference(B))

    df = pd.DataFrame([TP,FP,FN]).rename({0:"Counts"}, axis=1)
    df.index = ["TP", "FP", "FN"]
    
    return df

In [4]:
def GetContingencyTable(reference:pd.DataFrame, n:int=10000) -> pd.DataFrame:
    
    '''
    
    This function get the contingency table (TP, FP, FN) from a random network obtained
    
    Parameters
    ----------
    
    reference: pandas DataFrame
        Network containing columns "TF" (Transcription factors) and TG (Target gene). Every pair of values 
        TF-TG will describe an interaction
    
    n: int; default 10000
        Number of random networks to retrieve (experiments)
        
    Returns
    -------
    
    pd.DataFrame: A pandas data frame containing the counts of TP, FP, FN for every experiment
    
    '''

    print(f"Getting contingency table with n={n}")
    
    random_contingency_table = pd.DataFrame(dtype=int, index=["TP","FP","FN"])
    number_of_interactions = New_network.shape[0]

    start = time()

    for experiment in range(n):

        print(f"\tExperiment: {experiment + 1}", end="\r")
        experiment = "rand_" + str(experiment)
        random_df = reference.copy()

        # If we do not transfor into a list, then the data frame matches the rows by index
        # and the shuffle effect is lost 
    
        random_df["TF"] = random_df["TF"].sample(number_of_interactions).to_list()

        Interactions_sample = set(random_df[["TF","TG"]].apply(" | ".join, axis=1))
        metrics = BinaryMetrics(Interactions_known, Interactions_sample)["Counts"]
        random_contingency_table = pd.concat([random_contingency_table, metrics.rename(experiment)], axis=1)

    end = time()
    print(f"\tExperiment: {n}")
    print(f"\tThis functions took {end - start} secs to run")
    
    return random_contingency_table

In [6]:
def FilterNetworks(graph:pd.DataFrame) -> "(known = pd.DataFrame, new = pd.DataFrame)":
    
    '''
    
    This function filters the "Known" and "New" (a.k.a inferred) interactions between two networks given
    a specific format
    
    Parameters
    
    graph: pandas dataFrame
        A pandas dataframe containing the whole network
        
    Returns
    -------
    
    (known = pd.DataFrame, new = pd.DataFrame): A two dimention tuple containing the known interactions
        and the new interactions filtered
    
    '''
    
    print("Filtering networks")
    
    know_net = graph.query("STATUS == 'Known'")
    mask_new_cond_1 = graph["STATUS"] == "New"
    mask_new_cond_2 = (graph["STATUS"] == "Known") & ((graph["ORG_REF"] != "NOT_REFR_ORG") | (graph["TU_UNIT"] != "NOT_TU_REFER")) 

    new_net = graph[mask_new_cond_1 | mask_new_cond_2]

    return (know_net, new_net)

In [7]:
def ListDirectory(path:str, filter:bool=False) -> "list":
    
    '''
    This function list the file names found inside a directory
    
    Parameters
    ----------
    
    path: str
        String describing the path of the target directory
    
    filter: str or RegEx, default: False
        String or RegEx instruction matching with the target file names
    
    Returns
    -------
    
    list: list containing the file names
    
    '''
    
    print(f"Path directory: {path}")
    print(f"Filter: {filter}\n")
    
    file_names = pd.Series(os.listdir(path))
    
    if filter:
        file_names = file_names[file_names.str.contains(f"{filter}")]
    
    return list(file_names)

In [8]:
def LoadParserNet(file:str, **kwargs) -> "pd.DataFrame":
    
    '''
    This function load into a pandas DataFrame the specified network
    
    Parameters
    ----------
    
    file: str
        String describing the file name
        
    **kwargs:
        Aditional named keys for the pd.read_csv object
    
    
    Returns
    -------
    
    pd.DataFrame: Pandas DataFrame with the network loaded
        
    '''
    
    print(f"Loading file: {file}")
    
    return pd.read_csv(file, **kwargs)

In [9]:
if __name__ == "__main__":

    # Listing and saving file names of the networks
    network_path = "../IIMAS_V4/net/results_plus_TU"
    network_file_names = ListDirectory(network_path, filter="^GCF")

    # Same arguments for all files
    arguments = {"sep":"\t",
            "header":None,
            "usecols":[1,2,3,4,8],
            "index_col":None,
            "names":["TF","TG","STATUS","ORG_REF", "TU_UNIT"]}

    for net in network_file_names:
    
        real_path = os.path.join(network_path, net)
        plain_graph = LoadParserNet(real_path, **arguments)
    
        # Getting the inferred interactions and the known interactions
        New_network, Known_network = FilterNetworks(plain_graph)
    
        # Getting the contingency table of the observed values of the inferred network and the network used
        # as a reference
        Interactions_new = set((New_network[["TF","TG"]].apply(" | ".join, axis=1)))
        Interactions_known = set((Known_network[["TF","TG"]].apply(" | ".join, axis=1)))
    
        Known_versus_inferred_metrics = BinaryMetrics(Interactions_known, Interactions_new)
    
        # Getting contingency table and expected values of random networks based on the inferred network
        contingency = GetContingencyTable(New_network, 10000)
        probabilities = GetExpectedProbability(contingency, 1)
    
        # Doing the test chi and G for goodness of fit
    
        Observed = Known_versus_inferred_metrics.values.reshape(-1,)
        Expected = (probabilities * Observed.sum()).values.reshape(-1,)
        
        # G-test directly
        G_d, p_value_G_d = stats.power_divergence(Observed, Expected, lambda_ = 0)
        
        print("Tests:")
        print(f"\tGd:{G_d}; p:{p_value_G_d}")
        
        print("")

Path directory: ../IIMAS_V4/net/results_plus_TU
Filter: ^GCF

Loading file: ../IIMAS_V4/net/results_plus_TU/GCF_000195955.2_ASM19595v2_M_tuberculosis_H37Rv_genomic_extended_network_plus_TU.txt
Filtering networks
Getting contingency table with n=10000
	Experiment: 10000
	This functions took 1172.5626637935638 secs to run
Tests:
	chi:19851.929906928002; p:0.0
	G:8508.405559605082; p:0.0
	Gd:8508.405559605082; p:0.0

Loading file: ../IIMAS_V4/net/results_plus_TU/GCF_000009045.1_ASM904v1_B_subtilis_168_genomic_extended_network_plus_TU.txt
Filtering networks
Getting contingency table with n=10000
	Experiment: 10000
	This functions took 867.7725138664246 secs to run
Tests:
	chi:36266.04720884251; p:0.0
	G:11012.8756506352; p:0.0
	Gd:11012.8756506352; p:0.0

Loading file: ../IIMAS_V4/net/results_plus_TU/GCF_000009645.1_ASM964v1_S_aureus_N315_genomic_extended_network_plus_TU.txt
Filtering networks
Getting contingency table with n=10000
	Experiment: 10000
	This functions took 811.0757734775543 